In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset#, load_metric
import numpy as np
from evaluate import load as load_metric

# Load the dataset
dataset = load_dataset("imdb")


In [15]:
dataset['train']['text'][2]

"If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br />"

In [14]:
dataset['train']['label'][2]

0

In [3]:
# Check GPU availability
if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("No GPU available, using CPU instead.")

No GPU available, using CPU instead.


In [13]:

# Split the dataset
train_dataset = dataset['train'].select(range(10))
test_dataset = dataset['test'].select(range(10))

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format to torch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Freeze all layers except the last layer of the encoder
for name, param in model.named_parameters():
    if 'bert.encoder.layer.11' not in name:  
        # Change '11' if using a different BERT model variant with more layers
        param.requires_grad = False

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
   # report_to=None
)

# Load the evaluation metric
metric = load_metric("accuracy")

# Define the compute_metrics function
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = torch.argmax(logits, dim=-1)
#     return metric.compute(predictions=predictions, references=labels)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # Ensure logits are a tensor
    if isinstance(logits, np.ndarray):
        logits = torch.tensor(logits)
        
    # Ensure labels are a tensor
    if isinstance(labels, np.ndarray):
        labels = torch.tensor(labels)
        
    # Use argmax with the tensor
    predictions = torch.argmax(logits, dim=-1)
    
    return metric.compute(predictions=predictions, references=labels)
    
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Ray Ge\anaconda3\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.6263046264648438, 'eval_accuracy': 1.0, 'eval_runtime': 28.308, 'eval_samples_per_second': 0.353, 'eval_steps_per_second': 0.071, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.6257013082504272, 'eval_accuracy': 1.0, 'eval_runtime': 28.5796, 'eval_samples_per_second': 0.35, 'eval_steps_per_second': 0.07, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.6246300339698792, 'eval_accuracy': 1.0, 'eval_runtime': 26.9175, 'eval_samples_per_second': 0.372, 'eval_steps_per_second': 0.074, 'epoch': 3.0}
{'train_runtime': 176.9528, 'train_samples_per_second': 0.17, 'train_steps_per_second': 0.034, 'train_loss': 0.6528035004933676, 'epoch': 3.0}


  0%|          | 0/2 [00:00<?, ?it/s]

Evaluation Results: {'eval_loss': 0.6246300339698792, 'eval_accuracy': 1.0, 'eval_runtime': 27.0798, 'eval_samples_per_second': 0.369, 'eval_steps_per_second': 0.074, 'epoch': 3.0}
